# Preprocessing for ML in Python
1. Intro to Data Preprocessing
2. Standardizing Data
3. Feature Engineering
4. Selecting features for modeling
5. Putting it all together

# 1. Intro to Data Preprocessing
- after cleaning and EDA
- = prepping data for modeling

Pandas review:
df.columns
df.dtypes
df.describe()

In [ ]:
# Remove missing data

# Drop all rows
df.dropna()

# Drop specific rows (default rows) using index labels
df.drop([1,2,3])

# Drop specific columns
df.drop("A", axis=1)

# Subset based on value
df[df["B"] == 7]

# Count NaN in column "B"
df["B"].isnull().sum()

# Filter out NaN in "B" column
df[df["B"].notnull()]


## 1.1 

## 1.2 

## 1.3 

## 1.4 

# 2. Standardizing Data

## 2.1 

## 2.2 

## 2.3 

## 2.4 

# 3. Feature Engineering

## 3.1 

## 3.2 

## 3.3 

## 3.4 

# 4. Selecting features for modeling

## 4.1 

## 4.2 

## 4.3 

## 4.4 

# 5. Putting it all together

## 5.1 

## 5.2 

## 5.3 

## 5.4 